# Long/short portfolio construction model trading strategy example

This is an example notebook how to construct a momentum based portfolio construction strategy
using [Trading Strategy framework](https://tradingstrategy.ai/docs/) and backtest it for DeFi tokens.

This backtest uses alpha model approach where each trading pair has a signal
and based on the signal strenghts we construct new portfolio weightings
for the upcoming week.


## Strategy parameter set up

Set up the parameters used in in this strategy backtest study.

- Backtested blockchain, exchange and trading pair
- Backtesting period
- Strategy parameters for EMA crossovers

### Strategy logic and trade decisions

Strategy logic and parameters are available as a separate [Python module](../strategies/long-short-momentum.py).


In [1]:
import datetime

from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module

import logging
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()


strategies_folder = get_default_strategies_path()
strategy_path = strategies_folder / "long-short-momentum.py"

## Load strategy and its universe

strategy_mod = read_strategy_module(strategy_path)

print(f"Backtesting for strategy engine version {strategy_mod.trading_strategy_engine_version}")

ImportError: cannot import name 'make_clickable' from 'tradingstrategy.utils.jupyter' (/Users/moo/Library/Caches/pypoetry/virtualenvs/web3-ethereum-defi-r0lBdNCP-py3.10/lib/python3.10/site-packages/tradingstrategy/utils/jupyter.py)

## Set up the market data client

The [Trading Strategy market data client](https://tradingstrategy.ai/docs/programming/api/client/index.html)
is the Python library responsible for managing the data feeds needed to run the backtest.None

We set up the market data client with an API key.

[If you do not have an API key yet, you can register one](https://tradingstrategy.ai/trading-view/backtesting).

In [ ]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Load the strategy universe and analyse what trading pairs the strategy code gives for us.

In [ ]:
from tradeexecutor.strategy.execution_context import notebook_execution_context

# contains start_at, end_at backtesting range
universe_options = strategy_mod.get_universe_options()

universe = strategy_mod.create_trading_universe(
    datetime.datetime.utcnow(),
    client,
    notebook_execution_context,
    universe_options,
)

## Available trading universe

Inspect trading pairs and lending markets we are going to backtest.


In [ ]:
from IPython.display import HTML

from tradeexecutor.analysis.universe import analyse_long_short_universe
from tradingstrategy.utils.jupyter import format_links_for_html_output

df = analyse_long_short_universe(universe)
df = format_links_for_html_output(df, ("Price data page", "Lending rate page",))

display(HTML(df.to_html(escape=False)))

## Run backtest

Run backtest using giving trading universe and strategy function.

- Running the backtest outputs `state` object that contains all the information
on the backtesting position and trades.
- The trade execution engine will download the necessary datasets to run the backtest.
  The datasets may be large, several gigabytes.


In [ ]:
from tradeexecutor.backtest.backtest_runner import setup_backtest, run_backtest

setup = setup_backtest(
    strategy_path,
)

state, universe, debug_dump = run_backtest(setup, client)

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

## Examine backtest results

Examine `state` that contains 
- All actions the trade executor took
- Visualisation and diagnostics data associated with the actity

We plot out a chart that shows
- The price action
- When the strategy made buys or sells

In [ ]:
print(f"Positions taken: {len(list(state.portfolio.get_all_positions()))}")
print(f"Trades made: {len(list(state.portfolio.get_all_trades()))}")

## Benchmarking the strategy performance

Here we benchmark the strategy performance against some baseline scenarios.

- Buy and hold ETH
- Buy and hold US Dollar (do nothing)


In [ ]:
from tradingstrategy.chain import ChainId
from tradeexecutor.visual.benchmark import visualise_benchmark

eth_usd = universe.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WETH", "USDC"))
eth_candles = universe.universe.candles.get_candles_by_pair(eth_usd.internal_id)
eth_price = eth_candles["close"]

fig = visualise_benchmark(
    state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_deposit(),
    buy_and_hold_asset_name="ETH",
    buy_and_hold_price_series=eth_price,
    start_at=universe_options.start_at,
    end_at=universe_options.end_at,
)

fig.show()

## Analysing  the strategy success

Here we calculate statistics on how well the strategy performed.

- Won/lost trades
- Timeline of taken positions with color coding of trade performance


In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

### Strategy summary

Overview of strategy performance



In [ ]:
import pandas as pd

summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

### Alpha model timeline analysis

We display a table that shows how alpha model performermed over time.

- Events are in the timeline order

- Display portfolio weightings at each strategy cycle

- Show performance of individidual positions taken during this cycle

- Stop loss events are shown as their own rows

The visualisation will have the a column for each asset being traded. The cell of a column contains the following elements

- Asset **price change** (up/down) since the last strategy cycle

- **Price** of the asset at the start of the strategy cycle

- **Weight** % as the part of the portfolio that was chosen for the next cycle

- **Open** value of the position as USD

- **Adjust** value of the position as USD -  to get to the target weight - how much asset needs to be bought or sold

- **Close** the remaining value of the position as USD 

- **Profit**: The life time profit of the position so far

Position opening and closing rules

- A position may contain multiple trades

- Unlike on some exchanges, we adjust existing positions by increasing or decreasing them - a trade can count against existing position

- Posions are opened if there are no existing holdings of an asset

- The position is adjusted over the time

- The position is closed when the remaining assets of the position are sold

In [ ]:
from tradeexecutor.analysis.alpha_model_analyser import render_alpha_model_plotly_table, create_alpha_model_timeline_all_assets

#timeline = create_alpha_model_timeline_all_assets(state, universe, new_line="<br>")
#figure, table = render_alpha_model_plotly_table(timeline)
#
#figure.update_layout(
#    autosize=True,
#    height=800,)
#
# display(figure)

## Finishing notes

Print out a line to signal the notebook finished the execution successfully.

In [ ]:
print("All ok")